In [6]:
%matplotlib inline
%load_ext autoreload
%autoreload 2
import sys, os
sys.path.append('..')

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [32]:
import inspect
from io import StringIO

from epypes import dag

In [8]:
g = dag.Digraph(range(10), [(0, 1), (5, 4), (5, 6)])

In [9]:
g.reverse()._adj

{0: [], 1: [0], 2: [], 3: [], 4: [5], 5: [], 6: [5], 7: [], 8: [], 9: []}

In [34]:
def plus(a, b):
    return a + b

def triple(x):
    return 3 * x

cg = dag.ComputationalGraph([plus, triple], ['a', 'b', 's', 'res'])
cg.add_edge('a', 'plus')
cg.add_edge('b', 'plus')
cg.add_edge('plus', 's')
cg.add_edge('s', 'triple')
cg.add_edge('triple', 'res')

cg.do_dfo()

print(cg._G._adj)

for v in cg._dfo.reverse_postorder:
    print(v, cg._G.adj(v))

Looking at vertex triple
dfs-start triple
triple {'res': False, 'b': False, 'a': False, 'triple': True, 'plus': False, 's': False} ['res'] ['res']
res is not marked
dfs-start res
res {'res': True, 'b': False, 'a': False, 'triple': True, 'plus': False, 's': False} [] []
dfs-end res
dfs-end triple
Looking at vertex plus
dfs-start plus
plus {'res': True, 'b': False, 'a': False, 'triple': True, 'plus': True, 's': False} ['s'] ['s']
s is not marked
dfs-start s
s {'res': True, 'b': False, 'a': False, 'triple': True, 'plus': True, 's': True} ['triple'] ['triple']
dfs-end s
dfs-end plus
Looking at vertex a
dfs-start a
a {'res': True, 'b': False, 'a': True, 'triple': True, 'plus': True, 's': True} ['plus'] ['plus']
dfs-end a
Looking at vertex res
Looking at vertex b
dfs-start b
b {'res': True, 'b': True, 'a': True, 'triple': True, 'plus': True, 's': True} ['plus'] ['plus']
dfs-end b
Looking at vertex s
{'res': [], 'b': ['plus'], 'a': ['plus'], 'triple': ['res'], 'plus': ['s'], 's': ['triple']}


In [35]:
cg._dfo.postorder, cg._G.vertices

(['res', 'triple', 's', 'plus', 'a', 'b'],
 {'a', 'b', 'plus', 'res', 's', 'triple'})

In [21]:
cg._functions

{'plus': <function __main__.plus>, 'triple': <function __main__.triple>}

In [12]:
content_tinyDAG = '''
13
15
2 3 
0 6 
0 1 
2 0 
11 12  
9 12  
9 10  
9 11 
3 5 
8 7 
5 4 
0 5 
6 4 
6 9 
7 6
'''

In [13]:
content_tinyDG = '''
13
22
 4  2
 2  3
 3  2
 6  0
 0  1
 2  0
11 12
12  9
 9 10
 9 11
 7  9
10 12
11  4
 4  3
 3  5
 6  8
 8  6
 5  4
 0  5
 6  4
 6  9
 7  6
'''

In [14]:
from epypes import dag

def read_graph(buf):
    buf.readline()
    num_vertices = int(buf.readline().strip())
    num_edges = int(buf.readline().strip())
    
    edges = []
    for pair in buf:
        a, b = pair.strip().split()
        edges.append((int(a), int(b)))
    
    return num_vertices, num_edges, edges
        
        
buf = StringIO(content_tinyDG)
num_vertices, num_edges, edges = read_graph(buf)

tinyDG = dag.Digraph(range(num_vertices), edges)

dc = dag.DirectedCycle(tinyDG)

In [31]:
from epypes import dag
        
num_vertices, num_edges, edges = read_graph(StringIO(content_tinyDAG))

tinyDAG = dag.Digraph(range(num_vertices), edges)

dc = dag.DirectedCycle(tinyDAG)
dfo = dag.DepthFirstOrder(tinyDAG)

Looking at vertex 0
Looking at vertex 1
Looking at vertex 2
Looking at vertex 3
Looking at vertex 4
Looking at vertex 5
Looking at vertex 6
Looking at vertex 7
Looking at vertex 8
Looking at vertex 9
Looking at vertex 10
Looking at vertex 11
Looking at vertex 12
Looking at vertex 0
dfs-start 0
0 {0: True, 1: False, 2: False, 3: False, 4: False, 5: False, 6: False, 7: False, 8: False, 9: False, 10: False, 11: False, 12: False} [6, 1, 5] [6, 1, 5]
6 is not marked
dfs-start 6
6 {0: True, 1: False, 2: False, 3: False, 4: False, 5: False, 6: True, 7: False, 8: False, 9: False, 10: False, 11: False, 12: False} [4, 9] [4, 9]
4 is not marked
dfs-start 4
4 {0: True, 1: False, 2: False, 3: False, 4: True, 5: False, 6: True, 7: False, 8: False, 9: False, 10: False, 11: False, 12: False} [] []
dfs-end 4
9 is not marked
dfs-start 9
9 {0: True, 1: False, 2: False, 3: False, 4: True, 5: False, 6: True, 7: False, 8: False, 9: True, 10: False, 11: False, 12: False} [12, 10, 11] [12, 10, 11]
12 is not m

In [17]:
for v in dfo.reverse_postorder:
    print(v, tinyDAG.adj(v))

8 [7]
7 [6]
2 [3, 0]
3 [5]
0 [6, 1, 5]
5 [4]
1 []
6 [4, 9]
9 [12, 10, 11]
11 [12]
10 []
12 []
4 []
